In [1]:
import re
from pyspark.sql.functions import regexp_extract, col

In [2]:
from pyspark.sql.session import SparkSession

In [3]:
spark=SparkSession.builder.appName("Module 5 SparkSession take").getOrCreate()

In [4]:
access_logs_DF = spark.read.text("/user/edureka_524533/Datasets/access.clean.log")

In [5]:
access_logs_DF.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
access_logs_DF.count()

2338006

In [7]:
#Extract time stamps
ts_pattern = r'\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}'

In [8]:
result = access_logs_DF.withColumn('Timestamp', regexp_extract(col('value'), ts_pattern, 0))

In [9]:
dt_pattern = r'\d{2}/\w{3}/\d{4}'

In [10]:
result = result.withColumn('Date', regexp_extract(col('TimeStamp'), dt_pattern, 0))

In [11]:
#Extract Host Name
ht_pattern=r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'

In [12]:
result = result.withColumn('Host', regexp_extract(col('value'), ht_pattern, 0))

In [13]:
#Extract Bytes
byt_pattern=r'\s\d{4,}\s'

In [14]:
result = result.withColumn('Bytes', regexp_extract(col('value'), byt_pattern, 0))

In [15]:
meth_pattern =r'\s[A-Z]{3,}\s'

In [16]:
result = result.withColumn('Method', regexp_extract(col('value'), meth_pattern, 0))

In [17]:
Url_pattern = r'\s/(\S+)'

In [18]:
result = result.withColumn('URL', regexp_extract(col('value'), Url_pattern, 0))

In [19]:
#Extract Status Codes from each
status_pattern = r'\s(\d{3})\s'

In [20]:
result = result.withColumn('StatusCode', regexp_extract(col('value'), status_pattern, 1))

In [21]:
result.show()

+--------------------+--------------------+-----------+---------------+------+------+--------------------+----------+
|               value|           Timestamp|       Date|           Host| Bytes|Method|                 URL|StatusCode|
+--------------------+--------------------+-----------+---------------+------+------+--------------------+----------+
|109.169.248.247 -...|12/Dec/2015:18:25:11|12/Dec/2015|109.169.248.247| 4263 |  GET |     /administrator/|       200|
|109.169.248.247 -...|12/Dec/2015:18:25:11|12/Dec/2015|109.169.248.247| 4494 | POST | /administrator/i...|       200|
|46.72.177.4 - - [...|12/Dec/2015:18:31:08|12/Dec/2015|    46.72.177.4| 4263 |  GET |     /administrator/|       200|
|46.72.177.4 - - [...|12/Dec/2015:18:31:08|12/Dec/2015|    46.72.177.4| 4494 | POST | /administrator/i...|       200|
|83.167.113.100 - ...|12/Dec/2015:18:31:25|12/Dec/2015| 83.167.113.100| 4263 |  GET |     /administrator/|       200|
|83.167.113.100 - ...|12/Dec/2015:18:31:25|12/Dec/2015| 

In [22]:
# Verify there are no null columns in the original dataset.
bad_rows_df = result.filter(result['Host'].isNull()| 
                             result['Timestamp'].isNull() | 
                             result['Method'].isNull() |
                             result['StatusCode'].isNull() |
                             result['Bytes'].isNull()|
                             result['Date'].isNull())

In [38]:
bad_rows_df.show()

+-----+---------+----+----+-----+------+---+----------+
|value|Timestamp|Date|Host|Bytes|Method|URL|StatusCode|
+-----+---------+----+----+-----+------+---+----------+
+-----+---------+----+----+-----+------+---+----------+



In [23]:
bad_rows_df.count()

0

In [24]:
#Describe which HTTP status values appear in data and how many
StatusCodeDF = result.select(result['StatusCode'])

In [25]:
StatusCodeDF=StatusCodeDF.dropDuplicates()

In [26]:
StatusCodeDF.show()

+----------+
|StatusCode|
+----------+
|       200|
|       406|
|       401|
|       206|
|       501|
|       404|
|       403|
|       412|
|       303|
|       500|
|       304|
|       405|
|       400|
|       301|
+----------+



In [27]:
StatusCodeDF.count()

14

In [28]:
#How many unique hosts are there in the entire log and their average request
HostsDF = result.select(result['Host'],result['Bytes'])

In [29]:
from pyspark.sql import functions as F

In [30]:
HostsDF = HostsDF.groupby('Host').agg(F.mean('Bytes'),F.count('Bytes').alias('AverageRequests'))

In [31]:
HostsDF.show()

+--------------+------------------+---------------+
|          Host|        avg(Bytes)|AverageRequests|
+--------------+------------------+---------------+
|   46.72.177.4|            4378.5|              8|
| 194.48.218.78|            4378.5|              2|
| 31.181.253.16|            4378.5|              2|
|  37.112.46.76|            4378.5|              2|
| 95.107.90.225|            4378.5|              2|
|  5.138.58.118|            4378.5|              2|
|95.188.228.228|            4378.5|              2|
|  66.7.119.112|          887508.0|              1|
| 145.255.2.176|            4378.5|              4|
| 176.59.208.95|            4378.5|              2|
| 62.133.162.65|            4378.5|              4|
| 95.29.129.235|            4378.5|              2|
|  66.249.64.64|10022.774193548386|             41|
| 207.46.13.165| 8839.333333333334|              6|
| 180.76.15.162|29545.735294117647|             75|
|  37.139.52.40|         1396107.0|             16|
| 89.144.209

In [32]:
HostsDF.count()

40836

In [33]:
# Find out how many 404 HTTP codes are in access logs
result.show()

+--------------------+--------------------+-----------+---------------+------+------+--------------------+----------+
|               value|           Timestamp|       Date|           Host| Bytes|Method|                 URL|StatusCode|
+--------------------+--------------------+-----------+---------------+------+------+--------------------+----------+
|109.169.248.247 -...|12/Dec/2015:18:25:11|12/Dec/2015|109.169.248.247| 4263 |  GET |     /administrator/|       200|
|109.169.248.247 -...|12/Dec/2015:18:25:11|12/Dec/2015|109.169.248.247| 4494 | POST | /administrator/i...|       200|
|46.72.177.4 - - [...|12/Dec/2015:18:31:08|12/Dec/2015|    46.72.177.4| 4263 |  GET |     /administrator/|       200|
|46.72.177.4 - - [...|12/Dec/2015:18:31:08|12/Dec/2015|    46.72.177.4| 4494 | POST | /administrator/i...|       200|
|83.167.113.100 - ...|12/Dec/2015:18:31:25|12/Dec/2015| 83.167.113.100| 4263 |  GET |     /administrator/|       200|
|83.167.113.100 - ...|12/Dec/2015:18:31:25|12/Dec/2015| 

In [36]:
Count404 = result.where(result['StatusCode']=='404').count()

In [37]:
Count404

227101